### Llama모델 학습
### 사용환경: Colab

In [ ]:
!pip install pyfiglet
from pyfiglet import Figlet
f = Figlet(font='slant')
print(f.renderText('HELLO SILVERPRIZE'))

### 키 파일 생성

In [ ]:

import json
from google.colab import userdata

data = {
  "type": "service_account",
  "project_id": "llama-433214",
  "private_key_id": userdata.get("private_key_id"),
  "private_key": userdata.get("private_key").replace("\\n", "\n"),
  "client_email": "storage-manager@llama-433214.iam.gserviceaccount.com",
  "client_id": "112540232109707769744",
  "auth_uri": "https://accounts.google.com/o/oauth2/auth",
  "token_uri": "https://oauth2.googleapis.com/token",
  "auth_provider_x509_cert_url": "https://www.googleapis.com/oauth2/v1/certs",
  "client_x509_cert_url": "https://www.googleapis.com/robot/v1/metadata/x509/storage-manager%40llama-433214.iam.gserviceaccount.com",
  "universe_domain": "googleapis.com"
}

json_file_path = "key.json"
with open(json_file_path, "w") as json_file:
    json.dump(data, json_file)

### Google Auth

In [ ]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "./key.json"